<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/deepseek_vl2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL2.git
%cd DeepSeek-VL2

Cloning into 'DeepSeek-VL2'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 211 (delta 61), reused 40 (delta 40), pack-reused 129 (from 2)
Receiving objects: 100% (211/211), 31.05 MiB | 19.76 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/DeepSeek-VL2


In [3]:
!ls

deepseek_env  DeepSeek_VL2_paper.pdf  inference.py  LICENSE-MODEL  pyproject.toml  requirements.txt
deepseek_vl2  images		      LICENSE-CODE  Makefile	   README.md	   web_demo.py


In [4]:
!python -m venv deepseek_env
!source deepseek_env/bin/activate  # On Windows: deepseek_env\Scripts\activate

Error: Command '['/content/DeepSeek-VL2/deepseek_env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: deepseek_env/bin/activate: No such file or directory


In [ ]:
import torch
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images

# Specify the model path
model_path = "deepseek-ai/deepseek-vl2-tiny"
processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = processor.tokenizer

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
model = model.to(torch.bfloat16).cuda().eval()

# Load and process the image
image_path = "./images/sample_image.jpg"
images = load_pil_images([image_path])

# Prepare the conversation
conversation = [
    {
        "role": "<|User|>",
        "content": "<image>\nDescribe the image.",
        "images": images,
    },
    {"role": "<|Assistant|>", "content": ""},
]

# Process the conversation
inputs = processor(conversation, return_tensors="pt")
inputs = {k: v.to(torch.bfloat16).cuda() for k, v in inputs.items()}

# Generate the response
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)
